In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from IPython.display import clear_output          
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn import tree
from collections import Counter
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm

### Preparo y entreno el label encoder y el one hot encoder con la info de todos los avisos existentes

In [2]:
avisos_totales = pd.concat([pd.read_csv('datos_navent_fiuba/fiuba_6_avisos_detalle.csv', usecols=['idaviso', 'nombre_area']), pd.read_csv('fiuba_hasta_15_abril/fiuba_6_avisos_detalle.csv', usecols=['idaviso', 'nombre_area']), pd.read_csv('Fiuba desde 15 Abril/fiuba_6_avisos_detalle.csv', usecols=['idaviso', 'nombre_area']), pd.read_csv('fiuba_6_avisos_detalle_missing_nivel_laboral.csv', usecols=['idaviso', 'nombre_area'])])
avisos_totales.drop_duplicates(subset=['idaviso'], inplace=True)
avisos_totales.reset_index(drop=True, inplace=True)
avisos_totales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25288 entries, 0 to 25287
Data columns (total 2 columns):
idaviso        25288 non-null int64
nombre_area    25288 non-null object
dtypes: int64(1), object(1)
memory usage: 395.2+ KB


In [3]:
avisos_totales['nombre_area'].value_counts().size

188

- Label encoding para las 188 areas, y entreno el One Hot Encoding con esos 188 codigos (0-187)

In [4]:
#Fundamental usar para el label encoder una columna DF que contenga TODOS los posibles valores categoricos del nombre de area, es decir, lo saco del DF de TODOS los anuncios.
#Esto es para que si en un set no estan todos, igual los codifique de la misma forma. SI lo entreno con un set que no tiene todos, al ingresar un set con valores nuevos
#va a tirar error porque no tenia esos labels entrenados

#UNA VEZ ENTRENADO EL LABEL ENCODER, Y EL ONEHOTENCODER, USAR SIEMRE EL MISMO PARA EL RESTO DEL NOTEBOOK CODIFICAR LOS SETS DE DATA (TEST Y TRAIN) Y EL SET DE TEST100K

le = preprocessing.LabelEncoder()
avisos_totales['codigo_area'] = le.fit_transform(avisos_totales['nombre_area'].astype(str))

#PARA EL ONE HOT ENCODING, AVISARLE QUE LA CANTIDAD DE VALORES CATEGORICOS ES EL DE TODAS LAS CATEGORIAS EN EL DF DE TODOS LOS ANUNCIOS:188
enc = preprocessing.OneHotEncoder(188)   
enc.fit(avisos_totales['codigo_area'].values.reshape(-1,1))

avisos_totales.head()

,idaviso,nombre_area,codigo_area
0,8725750,Comercial,30
1,17903700,Salud,158
2,1000150677,Transporte,181
3,1000610287,Transporte,181
4,1000872556,Producción,143


### Cargo historial de postulaciones x usuario x area, ya encodeado con One Hot Encoding

In [5]:
users_ids_history = pd.read_csv('historial_cant_post_xuser_xarea.csv', dtype={i+1:'int16' for i in range(188)})
users_ids_history.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 346764 entries, 0 to 346763
Columns: 189 entries, idpostulante to 187
dtypes: int16(188), object(1)
memory usage: 127.0+ MB


### Cargo Dataset y test100k para operar

In [6]:
data_set_complete_v2 = pd.read_csv('data_set_complete_v2.csv', usecols=['sepostulo','idpostulante','veces_visto','sexo','edad','titulo','nombre_area',\
                                                                  'estudio_encurso','estudio_graduado','tipo_full-time','tipo_part-time','tipo_otro',\
                                                                  'zona_gba','zona_caba','nivel_senior','nivel_junior','nivel_otro','nivel_jefe','nivel_gerente','post_x_aviso','vistas_x_aviso'],\
                                dtype={'sepostulo':'int8', 'idpostulante':'object', 'veces_visto':'int16', 'sexo':'int8', 'edad':'int8', 'nombre_area':'category',\
                                       'estudio_encurso':'int8', 'estudio_graduado':'int8', 'tipo_full-time':'int8','tipo_part-time':'int8','tipo_otro':'int8',\
                                       'zona_gba':'int8','zona_caba':'int8','nivel_senior':'int8','nivel_junior':'int8','nivel_otro':'int8','nivel_jefe':'int8','nivel_gerente':'int8', 'post_x_aviso':'int32','vistas_x_aviso':'int32'})

data_set_complete_v2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11870121 entries, 0 to 11870120
Data columns (total 21 columns):
sepostulo           int8
idpostulante        object
veces_visto         int16
sexo                int8
edad                int8
titulo              object
nombre_area         category
estudio_encurso     int8
estudio_graduado    int8
tipo_full-time      int8
tipo_part-time      int8
tipo_otro           int8
zona_gba            int8
zona_caba           int8
nivel_senior        int8
nivel_junior        int8
nivel_otro          int8
nivel_jefe          int8
nivel_gerente       int8
post_x_aviso        int32
vistas_x_aviso      int32
dtypes: category(1), int16(1), int32(2), int8(15), object(2)
memory usage: 486.8+ MB


In [7]:
#Achico a 1 millon para maniabrabilidad en terminos de performance
data = data_set_complete_v2.sample(1000000)

del data_set_complete_v2

data.reset_index(drop=True, inplace=True)

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000000 entries, 0 to 2999999
Data columns (total 21 columns):
sepostulo           int8
idpostulante        object
veces_visto         int16
sexo                int8
edad                int8
titulo              object
nombre_area         category
estudio_encurso     int8
estudio_graduado    int8
tipo_full-time      int8
tipo_part-time      int8
tipo_otro           int8
zona_gba            int8
zona_caba           int8
nivel_senior        int8
nivel_junior        int8
nivel_otro          int8
nivel_jefe          int8
nivel_gerente       int8
post_x_aviso        int32
vistas_x_aviso      int32
dtypes: category(1), int16(1), int32(2), int8(15), object(2)
memory usage: 123.0+ MB


In [8]:
test100k = pd.read_csv('test100k_complete_v2.csv', usecols=['id','idpostulante','veces_visto','sexo','edad','titulo','nombre_area',\
                                                                  'estudio_encurso','estudio_graduado','tipo_full-time','tipo_part-time','tipo_otro',\
                                                                  'zona_gba','zona_caba','nivel_senior','nivel_junior','nivel_otro','nivel_jefe','nivel_gerente','post_x_aviso','vistas_x_aviso'],\
                                dtype={'id':'int64', 'idpostulante':'object', 'veces_visto':'int16', 'sexo':'int8', 'edad':'int8', 'nombre_area':'category',\
                                       'estudio_encurso':'int8', 'estudio_graduado':'int8', 'tipo_full-time':'int8','tipo_part-time':'int8','tipo_otro':'int8',\
                                       'zona_gba':'int8','zona_caba':'int8','nivel_senior':'int8','nivel_junior':'int8','nivel_otro':'int8','nivel_jefe':'int8','nivel_gerente':'int8','post_x_aviso':'int32','vistas_x_aviso':'int32'})

test100k.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 21 columns):
id                  100000 non-null int64
idpostulante        100000 non-null object
veces_visto         100000 non-null int16
sexo                100000 non-null int8
edad                100000 non-null int8
titulo              100000 non-null object
nombre_area         100000 non-null category
estudio_encurso     100000 non-null int8
estudio_graduado    100000 non-null int8
tipo_full-time      100000 non-null int8
tipo_part-time      100000 non-null int8
tipo_otro           100000 non-null int8
zona_gba            100000 non-null int8
zona_caba           100000 non-null int8
nivel_senior        100000 non-null int8
nivel_junior        100000 non-null int8
nivel_otro          100000 non-null int8
nivel_jefe          100000 non-null int8
nivel_gerente       100000 non-null int8
post_x_aviso        100000 non-null int32
vistas_x_aviso      100000 non-null int32
dtypes: category(

### Uso label encoding para nombre_area

In [9]:
data['nombre_area'] = (le.transform(data['nombre_area'].astype(str))).astype('int16')

In [10]:
test100k['nombre_area'] = (le.transform(test100k['nombre_area'].astype(str))).astype('int16')

### One Hot Encoding para nombre_area

In [11]:
encoded_areas = enc.transform(data['nombre_area'].values.reshape(-1,1))
encoded_areas_df = pd.DataFrame(encoded_areas.toarray()).astype('int8')

data = data.merge(encoded_areas_df, left_index=True, right_index=True)

del encoded_areas, encoded_areas_df

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000000 entries, 0 to 2999999
Columns: 209 entries, sepostulo to 187
dtypes: int16(2), int32(2), int8(203), object(2)
memory usage: 660.9+ MB


In [12]:
data.head()

,sepostulo,idpostulante,veces_visto,sexo,edad,titulo,nombre_area,estudio_encurso,estudio_graduado,tipo_full-time,...,178,179,180,181,182,183,184,185,186,187
0,0,bOJWLN9,0,0,26,ADMINISTRATIVO DE COMERCIO EXTERIOR,186,3,1,1,...,0,0,0,0,0,0,0,0,1,0
1,0,KBzokGX,0,1,27,Estudiantes en búsqueda de Primer Empleo (Vent...,26,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1,EzeqjKz,0,0,38,OPERADORES TELEFÓNICOS DE ATENCIÓN AL CLIENTE,14,4,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,5ZxpWk,0,0,38,Asistente de Marketing,120,3,0,1,...,0,0,0,0,0,0,0,0,0,0
4,0,eK0DPr,0,1,33,Lider de compras de insumos,34,0,1,1,...,0,0,0,0,0,0,0,0,0,0


In [13]:
encoded_areas = enc.transform(test100k['nombre_area'].values.reshape(-1,1))
encoded_areas_df = pd.DataFrame(encoded_areas.toarray()).astype('int8')

test100k = test100k.merge(encoded_areas_df, left_index=True, right_index=True)

del encoded_areas, encoded_areas_df

test100k.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Columns: 209 entries, id to 187
dtypes: int16(2), int32(2), int64(1), int8(202), object(2)
memory usage: 22.7+ MB


In [14]:
test100k.head()

,id,idpostulante,veces_visto,sexo,edad,titulo,nombre_area,estudio_encurso,estudio_graduado,tipo_full-time,...,178,179,180,181,182,183,184,185,186,187
0,0,6M9ZQR,0,0,42,Asistente Comercial,30,0,7,1,...,0,0,0,0,0,0,0,0,0,0
1,1,6v1xdL,0,1,30,Asistente Comercial,30,2,3,1,...,0,0,0,0,0,0,0,0,0,0
2,2,ezRKm9,0,0,36,Asistente Comercial,30,0,4,1,...,0,0,0,0,0,0,0,0,0,0
3,3,1Q35ej,0,1,68,Oracle Financials Sr. Application Developer,170,0,5,1,...,0,0,0,0,0,0,0,0,0,0
4,4,EAN4J6,0,0,32,Oracle Financials Sr. Application Developer,170,4,0,1,...,0,0,0,0,0,0,0,0,0,0


### Merge del historial con One Hot Coding

In [15]:
# Aca meto la info del historial (matriz de 188 con el one hot encoding)
data = data.merge((data['idpostulante'].to_frame().merge(users_ids_history, how='left', on='idpostulante').drop(columns='idpostulante').fillna(value=0).astype('int16', copy=False)), left_index=True, right_index=True)

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000000 entries, 0 to 2999999
Columns: 397 entries, sepostulo to 187
dtypes: int16(190), int32(2), int8(203), object(2)
memory usage: 1.7+ GB


In [16]:
data.head()

,sepostulo,idpostulante,veces_visto,sexo,edad,titulo,nombre_area,estudio_encurso,estudio_graduado,tipo_full-time,...,178,179,180,181,182,183,184,185,186,187
0,0,bOJWLN9,0,0,26,ADMINISTRATIVO DE COMERCIO EXTERIOR,186,3,1,1,...,0,0,0,0,0,0,0,4,0,0
1,0,KBzokGX,0,1,27,Estudiantes en búsqueda de Primer Empleo (Vent...,26,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1,EzeqjKz,0,0,38,OPERADORES TELEFÓNICOS DE ATENCIÓN AL CLIENTE,14,4,0,0,...,0,0,0,0,0,0,1,28,0,0
3,0,5ZxpWk,0,0,38,Asistente de Marketing,120,3,0,1,...,0,0,0,0,0,0,0,0,0,0
4,0,eK0DPr,0,1,33,Lider de compras de insumos,34,0,1,1,...,0,0,0,0,0,0,0,12,0,0


In [17]:
# Aca meto la info del historial (matriz de 188 con el one hot encoding)
test100k = test100k.merge((test100k['idpostulante'].to_frame().merge(users_ids_history, how='left', on='idpostulante').drop(columns='idpostulante').fillna(value=0).astype('int16', copy=False)), left_index=True, right_index=True)

test100k.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Columns: 397 entries, id to 187
dtypes: int16(190), int32(2), int64(1), int8(202), object(2)
memory usage: 58.6+ MB


In [18]:
test100k.head()

,id,idpostulante,veces_visto,sexo,edad,titulo,nombre_area,estudio_encurso,estudio_graduado,tipo_full-time,...,178,179,180,181,182,183,184,185,186,187
0,0,6M9ZQR,0,0,42,Asistente Comercial,30,0,7,1,...,0,0,0,0,0,0,0,0,0,0
1,1,6v1xdL,0,1,30,Asistente Comercial,30,2,3,1,...,0,0,0,0,0,0,0,4,0,0
2,2,ezRKm9,0,0,36,Asistente Comercial,30,0,4,1,...,0,0,0,0,0,0,0,0,0,0
3,3,1Q35ej,0,1,68,Oracle Financials Sr. Application Developer,170,0,5,1,...,0,0,0,0,0,0,0,0,0,0
4,4,EAN4J6,0,0,32,Oracle Financials Sr. Application Developer,170,4,0,1,...,0,0,0,0,0,0,0,0,0,0


### Pruebo con Random Forest

In [19]:
#Saco edad porque vimos que resta score
#del train_set, test_set, train_classes, test_classes
train_set, test_set, train_classes, test_classes = train_test_split(data.drop(columns=['edad','sepostulo','idpostulante', 'titulo']), data['sepostulo'], test_size=0.3)

In [38]:
clf = RandomForestClassifier(n_estimators=40, min_samples_leaf=7, min_samples_split=4, n_jobs=-1).fit(train_set, train_classes)
print('Test:', clf.score(test_set, test_classes), 'Entrenamiento:', clf.score(train_set, train_classes))

Test: 0.9022588888888889 Entrenamiento: 0.9179080952380952


In [39]:
test100k['sepostulo'] = clf.predict(test100k.drop(columns=['edad','id','idpostulante', 'titulo']))

In [40]:
test100k[['id','sepostulo']].to_csv('Resultados/2nuevasfeatures_historialint16_3M_n60.csv', index=False)

### GridSearch

In [31]:
#GRID SEARCH
parameters = {'n_estimators':[20,30,40],'min_samples_leaf':[5,6,7,8,9], 'min_samples_split':[2,3,4,5,6]}
rndforst = RandomForestClassifier(n_jobs=-1)
clfGS =  GridSearchCV(rndforst, parameters)
clfGS.fit(train_set, train_classes)
clfGS.best_params_ 

{'min_samples_leaf': 5, 'min_samples_split': 3, 'n_estimators': 40}

In [32]:
print('Test:', clfGS.score(test_set, test_classes), 'Entrenamiento:', clfGS.score(train_set, train_classes))

Test: 0.8946266666666667 Entrenamiento: 0.9184957142857143


In [35]:
test100k['sepostulo'] = clfGS.predict(test100k.drop(columns=['edad', 'sepostulo','id','idpostulante', 'titulo']))

In [36]:
test100k[['id','sepostulo']].to_csv('Resultados/grid_search_2nuevasfeatures.csv', index=False)

### Ultimos resultados

- Con post_x_aviso, vistas_x_aviso, historial correcto int16
    - GridSearch 1M rows
        - parameters = {'n_estimators':[20,30,40],'min_samples_leaf':[5,6,7,8,9], 'min_samples_split':[2,3,4,5,6]}
        - .best_params_ {'min_samples_leaf': 5, 'min_samples_split': 3, 'n_estimators': 40}
        - Test: 0.8946266666666667 Entrenamiento: 0.9184957142857143
        - Kaggle:
        - test100k[['id','sepostulo']].to_csv('Resultados/grid_search_2nuevasfeatures.csv', index=False)
    - 3M rows
        - (n_estimators=40, min_samples_leaf=7, min_samples_split=4, n_jobs=-1)
            - Test: 0.9018566666666666 Entrenamiento: 0.9173366666666667
            - test100k[['id','sepostulo']].to_csv('Resultados/2nuevasfeatures_historialint16_3M.csv', index=False)
        - (n_estimators=60, min_samples_leaf=7, min_samples_split=4, n_jobs=-1)
            - Test: 0.9022588888888889 Entrenamiento: 0.9179080952380952
            - test100k[['id','sepostulo']].to_csv('Resultados/2nuevasfeatures_historialint16_3M_n60.csv', index=False)
        - GS params (n_estimators=40, min_samples_leaf=5, min_samples_split=3, n_jobs=-1)
            - Test: 0.9046566666666667 Entrenamiento: 0.9254180952380953
            - test100k[['id','sepostulo']].to_csv('Resultados/2nuevasfeatures_historialint16_3M_conGSparam.csv', index=False)
        - GS params + 20n (n_estimators=60, min_samples_leaf=5, min_samples_split=3, n_jobs=-1)
            - Test: 0.9052088888888888 Entrenamiento: 0.9260247619047619
            - test100k[['id','sepostulo']].to_csv('Resultados/2nuevasfeatures_historialint16_3M_conGSparam_n60.csv', index=False)

### Resultados para ver aporte de vistas_x_aviso y post_x_aviso

- RandomForestClassifier(n_estimators=40, min_samples_leaf=7, min_samples_split=4, n_jobs=-1)
    - Sin post_x_aviso 
        - Test: 0.8871733333333334 Entrenamiento: 0.90157
    - Con post_x_aviso
        - Test: 0.8901833333333333 Entrenamiento: 0.9061214285714285
            - Sin vistas_x_aviso
                - Test: 0.8898566666666666 Entrenamiento: 0.9059657142857143
            - Con vistas_x_aviso
                - Test: 0.89138 Entrenamiento: 0.9097014285714285
                    - Con historial incorrecto
                        - Test: 0.89138 Entrenamiento: 0.9097014285714285
                    - Con historial correcto
                        - Test: Test: 0.8910966666666666 Entrenamiento: 0.90924
                    - Con historial mixto correcto vistas+post
                        - Test: 0.8286833333333333 Entrenamiento: 0.8529514285714286

### Resultados para ver la diferencia con el historial con error, el correcto con int16, y otros

- Sin post_x_aviso, vistas_x_aviso, edad
    - Con historiales negativos:
        - (n_estimators=40, min_samples_leaf=7, min_samples_split=4, n_jobs=-1)
            - Test: 0.8863633333333333 Entrenamiento: 0.9017185714285715
            - Kaggle: 0.86702

    - Con historial "pseudo-fix"
        - (n_estimators=40, min_samples_leaf=7, min_samples_split=4, n_jobs=-1)
            - Test: 0.8865466666666667 Entrenamiento: 0.9016171428571429
        - (n_estimators=30, min_samples_leaf=7, min_samples_split=4, n_jobs=-1)
            - Test: 0.88608 Entrenamiento: 0.90129
        - (n_estimators=40, min_samples_leaf=6, min_samples_split=5, n_jobs=-1)
            - Test: 0.88909 Entrenamiento: 0.9064142857142857    
            - Kaggle: 0.87545

    - Con historial correcto int16:
        - (n_estimators=40, min_samples_leaf=7, min_samples_split=4, n_jobs=-1)
            - Test: 0.8875633333333334 Entrenamiento: 0.9020371428571429
    - Con historial mixto int16 vistas+post
        - (n_estimators=40, min_samples_leaf=7, min_samples_split=4, n_jobs=-1)
            - Test: 0.8156833333333333 Entrenamiento: 0.8367371428571428 

### Resultados para ver diferencia con y sin One Hot Encoding (188 columnas en 1/0) de los labels (codificados de 0-187 por el LabelEncoder) de nombre_area

- Sin post_x_aviso, vistas_x_aviso, edad
    - Sin One Hot Encoding en nombre_areas
        - (n_estimators=40, min_samples_leaf=7, min_samples_split=4, n_jobs=-1)
            - Test: 0.8612533333333333 Entrenamiento: 0.8845785714285714
    - Con One Hot Encoding en nombres_areas
        - (n_estimators=40, min_samples_leaf=7, min_samples_split=4, n_jobs=-1)
            - Test: 0.8863633333333333 Entrenamiento: 0.9017185714285715

### Resultados para ver la variacion del score segun el tamanio del set a utilizar

- Sin post_x_aviso, vistas_x_aviso, edad
    - 350m rows
        - (n_estimators=30, min_samples_leaf=5, min_samples_split=4, n_jobs=-1)
            - Test: 0.876752380952381 Entrenamiento: 0.9021428571428571
        - Grid Search 
            - parameters = {'n_estimators':[30],'min_samples_leaf':[6,7,8,9], 'min_samples_split':[3,4,5,6]}
            - .best_params_ {'min_samples_leaf': 6, 'min_samples_split': 5}
            - Test: 0.8759047619047619 Entrenamiento: 0.8957755102040816
            - Kaggle: 0.86963
            - test100k[['id','sepostulo']].to_csv('Resultados/con_info_actualizada_v2_350msamples_gridserach.csv', index=False)
    - (n_estimators=30, min_samples_leaf=7, min_samples_split=4, n_jobs=-1)
        - 1M rows (lo de siempre)
            - Test: 0.8863633333333333 Entrenamiento: 0.9017185714285715
            - Kaggle: 0.86702
        - 2M rows
            - Test: 0.892795 Entrenamiento: 0.9068257142857142
            - Kaggle: 0.87659
            - test100k[['id','sepostulo']].to_csv('Resultados/con_info_actualizada_v2_OHEncod_Hist_RndTree_2Msamples.csv', index=False)
        - 3M rows
            - Test: 0.8969855555555556 Entrenamiento: 0.9096571428571428
            - Kaggle: 0.88024
            - test100k[['id','sepostulo']].to_csv('Resultados/con_info_actualizada_v2_OHEncod_Hist_RndTree_3Msamples.csv', index=False)
    - DecisionTreeClassifier()
        - Test: 0.8704633333333334 Entrenamiento: 0.9947728571428571
        - Kaggle: 0.84400
        - test100k[['id','sepostulo']].to_csv('Resultados/con_info_actualizada_v2_OHEncod_Hist_NewParameters_GridS_DecisTree.csv', index=False)

In [52]:
#Counter(" ".join(data["denominacion_empresa"].astype(str)).split()).most_common(150)

[('S.A.', 80594),
 ('Adecco', 69531),
 ('Grupo', 50929),
 ('SA', 47919),
 ('-Región', 44707),
 ('&', 37446),
 ('Manpower', 36582),
 ('Argentina', 36200),
 ('RANDSTAD', 34875),
 ('SRL', 34716),
 ('de', 32268),
 ('-', 31862),
 ('Gestión', 29407),
 ('Consultores', 27782),
 ('Office', 26439),
 ('Assistem', 24876),
 ('Servicios', 24065),
 ('GBA', 23887),
 ('Group', 23475),
 ('RRHH', 22330),
 ('Recursos', 20547),
 ('BBVA', 20144),
 ('Francés', 20144),
 ('Humanos', 19297),
 ('Farmacity', 19182),
 ('Pullmen', 17841),
 ('Empresarios', 17841),
 ('BAYTON', 16546),
 ('S.R.L.', 16358),
 ('S.A', 15879),
 ('y', 15613),
 ('RH', 14903),
 ('SUR', 14594),
 ('Región', 13908),
 ('Consultora', 12718),
 ('Suministra', 12585),
 ('Consulting', 12490),
 ('OESTE', 11951),
 ('NORTE', 11883),
 ('ARGENTINA', 11801),
 ('Suple', 11761),
 ('Complement', 10654),
 ('(holding)', 10654),
 ('Asociados', 10442),
 ('Laboral', 9937),
 ('Marketing', 9838),
 ('DE', 9795),
 ('en', 9597),
 ('Medical', 8785),
 ('Talentum', 8713),


### Probamos SVM

- SVM
    - LinearSVC()
        - Test: 0.7825833333333333 Entrenamiento: 0.7822107142857143
    - LinearSVC(C=1.4)
        - Test: 0.73895 Entrenamiento: 0.737025
    - LinearSVC(C=1.2)
        - Test: 0.7725916666666667 Entrenamiento: 0.7719678571428571
    - LinearSVC(C=0.8)
        - Test: 0.5775583333333333 Entrenamiento: 0.5793035714285715
    - LinearSVC(C=0.6)
        - 0.7768833333333334 Entrenamiento: 0.7756821428571429

In [20]:
small_data = data.sample(400000)
#del train_set, test_set, train_classes, test_classes
train_set, test_set, train_classes, test_classes = train_test_split(small_data.drop(columns=['edad','sepostulo','idpostulante', 'titulo']), small_data['sepostulo'], test_size=0.3)

In [23]:
svmclf = svm.LinearSVC().fit(train_set, train_classes)
print('Test:', svmclf.score(test_set, test_classes), 'Entrenamiento:', svmclf.score(train_set, train_classes))

Test: 0.7825833333333333 Entrenamiento: 0.7822107142857143


In [24]:
svmclf2 = svm.LinearSVC(C=1.2).fit(train_set, train_classes)
print('Test:', svmclf2.score(test_set, test_classes), 'Entrenamiento:', svmclf2.score(train_set, train_classes))

Test: 0.7725916666666667 Entrenamiento: 0.7719678571428571


In [25]:
svmclf3 = svm.LinearSVC(C=0.8).fit(train_set, train_classes)
print('Test:', svmclf3.score(test_set, test_classes), 'Entrenamiento:', svmclf3.score(train_set, train_classes))

Test: 0.5775583333333333 Entrenamiento: 0.5793035714285715


In [26]:
svmclf4 = svm.LinearSVC(C=0.6).fit(train_set, train_classes)
print('Test:', svmclf4.score(test_set, test_classes), 'Entrenamiento:', svmclf4.score(train_set, train_classes))

Test: 0.7768833333333334 Entrenamiento: 0.7756821428571429


In [27]:
svmclf5 = svm.LinearSVC(C=1.4).fit(train_set, train_classes)
print('Test:', svmclf5.score(test_set, test_classes), 'Entrenamiento:', svmclf5.score(train_set, train_classes))

Test: 0.73895 Entrenamiento: 0.737025


In [ ]:
#GRID SEARCH
parameters = {'n_estimators':[100,150,200], 'learning_rate':[1.0,0.8,0.6,1.2,1.4], 'min_samples_split':[3,4,5,6,7], 'min_samples_leaf':[1,5,7,10,15]}
gboost = GradientBoostingClassifier()
gboostclfGS = GridSearchCV(gboost, parameters)
gboostclfGS.fit(train_set, train_classes)
gboostclfGS.best_params_

In [ ]:
print('Test:', gboostclfGS.score(test_set, test_classes), 'Entrenamiento:', gboostclfGS.score(train_set, train_classes))

In [ ]:
knnclf = KNeighborsClassifier(n_jobs=-1, n_neighbors=7, algorithm='ball_tree').fit(train_set, train_classes)
print('Test:', knnclf.score(test_set, test_classes), 'Entrenamiento:', knnclf.score(train_set, train_classes))

In [ ]:
#GRID SEARCH

parameters = {'algorithm':('ball_tree', 'kd_tree', 'brute'), 'n_neighbors':[3,4,5,6,7,8,9,10]}
knnclf = KNeighborsClassifier(n_jobs=-1)
knnclfGS = GridSearchCV(knnclf, parameters)
knnclfGS.fit(train_set, train_classes)
knnclfGS.best_params_

In [ ]:
print('Test:', knnclfGS.score(test_set, test_classes), 'Entrenamiento:', knnclfGS.score(train_set, train_classes))

In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
logreg = LogisticRegression(max_iter=200, n_jobs=-1, solver='saga').fit(train_set, train_classes)
print('Test:',logreg.score(test_set, test_classes), 'Entrenamiento:', logreg.score(train_set, train_classes))

In [ ]:
neural=MLPClassifier().fit(train_set, train_classes)
print('Test:',neural.score(test_set, test_classes), 'Entrenamiento:', neural.score(train_set, train_classes))